In [1]:
import os
import pyodbc
import urllib
import sys
import pandas as pd
from pyspark import  SparkContext
from pyspark import SparkConf
from functools import reduce  # For Python 3.x
from pyspark.sql import SQLContext
from pyspark.sql import DataFrame
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from glob import glob
from os.path import normpath, basename
import shutil

print(sys.executable)

sqlContext.setConf("spark.sql.parquet.compression.codec", "snappy")

def save_calc_csv_to_parquet(csv_path, path_to_save):
    customSchema = StructType([StructField("EventSetId", LongType(), True),
    StructField("GeoId", LongType(), True), 
    StructField("PerilId", LongType(), True),    
    StructField("LobId", LongType(), False), 
    StructField("Simulation", LongType(), True), 
    StructField("EventId", LongType(), True),     
    StructField("EventDay", LongType(), True),         
    StructField("LayerAttrition", DoubleType(), True), 
    StructField("ReinstatementAttrition", DoubleType(), True)])
    
    df = sqlContext.read \
    .format('com.databricks.spark.csv') \
    .options(header='true') \
    .load(csv_path, schema = customSchema)
    
    df.write.parquet(path_to_save)
    
def save_catrader_csv_to_parquet(csv_path, path_to_save):
    customSchema = StructType([StructField("EventSetId", LongType(), True),
    StructField("GeoId", LongType(), True), 
    StructField("PerilId", LongType(), True),    
    StructField("LobId", LongType(), False), 
    StructField("Simulation", LongType(), True), 
    StructField("EventId", LongType(), True),     
    StructField("EventDay", LongType(), True),    
    StructField("Loss", DoubleType(), True)])
    
    df = sqlContext.read \
    .format('com.databricks.spark.csv') \
    .options(header='true') \
    .load(csv_path, schema = customSchema)
    
    
    #df = df.where(df['Simulation'] == 5830)
    #df.show()
    df.write.parquet(path_to_save)
                          


C:\ProgramData\Anaconda3\python.exe


In [ ]:
folders = glob("../../data/extracts/catraderimport/*/")

for folder in folders:
    eventset_id = basename(normpath(folder))
    csv_path = folder + str(eventset_id) + '.csv'
    path_to_save = folder + 'parquet'
    print(eventset_id)
    try:
        save_catrader_csv_to_parquet(csv_path, path_to_save)    
    except Exception as err:
            print(f'Error {eventset_id} {err}')

In [4]:
folder = '../data/extracts/v18/worldwide/105670/'

eventset_id = basename(normpath(folder))
csv_path = folder + str(eventset_id) + '.csv'
path_to_save = folder + 'parquet'
print(eventset_id)
try:
    save_catrader_csv_to_parquet(csv_path, path_to_save)    
except Exception as err:
        print(f'Error {eventset_id} {err}')

105670


In [ ]:
folders = glob("../data/extracts/v18/calculator/*/")

for folder in folders:
    eventset_id = basename(normpath(folder))
    csv_path = folder + str(eventset_id) + '.csv'
    path_to_save = folder + 'parquet'
    print(eventset_id)
    try:
        if (os.path.isdir(path_to_save)):
            shutil.rmtree(path_to_save)
        save_calc_csv_to_parquet(csv_path, path_to_save)    
    except Exception as err:
            print(f'Error {eventset_id} {err}')

In [5]:
df = sqlContext.read.load('../data/extracts/v18/worldwide/105670/parquet')
df = df.filter(df['Simulation'] == 5830)
df.write.parquet('../data/extracts/v18/worldwide/105670/biggest-year/parquet')